# Coherent info

In [1]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import MoebiusCode
from coherentinfo.linalg import (finite_field_gauss_jordan_elimination,
                                 finite_field_matrix_rank,
                                 finite_field_inverse,
                                 is_prime)

from coherentinfo.errormodel import ErrorModel, ErrorModelLindblad
# import galois
import timeit

In [8]:
length = 5
width = 35
p = 3
moebius_code = MoebiusCode(length=length, width=width, d=2 * p)
h_z = moebius_code.h_z
h_x = moebius_code.h_x
logical_x = moebius_code.logical_x
logical_z = moebius_code.logical_z

# Vertex syndrome

Now we want to understand how to construct a candidate vector given a certain vertex syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [9]:
for _ in range(1000):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_z @ error.T % (2 * p)
    candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_z @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

In [10]:
error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
error_diff = error - candidate_error
res_com = error_diff @ logical_z.T % (2 * p)
print(res_com == 0 or res_com == p)

True


# Plaquette Syndrome

Now we want to understand how to construct a candidate vector given a certain plaquette syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [11]:
for _ in range(100):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_x @ error.T % (2 * p)
    candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_x @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
error_diff = error - candidate_error
print(error_diff @ logical_x.T % (2 * p))

0


# Error model

In [12]:
gamma = 0.1
lindblad_em = ErrorModelLindblad(moebius_code.num_edges, 2 * p, gamma)

In [13]:
lindblad_em.get_probabilities()

array([1., 0., 0., 0., 0., 0.])

In [15]:
(lindblad_em.generate_error()).shape

(345,)

In [16]:
moebius_code.num_edges

345